# Automatización de descarga de datos de INEGI

Para generar un token para usar la API del INEGI se puede consulgar la siguiente [liga](https://www.inegi.org.mx/app/desarrolladores/generatoken/Usuarios/token_Verify)

In [ ]:
import requests
import numpy as np
import pandas as pd

token_1='8f108d9b-a840-3d04-f180-25d11dd88a7a'
url='https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/'+serie+'/es/00/true/'+es_BIE+'/2.0/'+token_1+'?type=json'

def descargar_inegi(serie,fecha_inicio,fecha_fin,token=token_1)
    url='https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/6207061409/es/0700/true/BISE/2.0/'+token_1+'?type=json'
    response = requests.get(url,headers=headers)
    status=response.status_code
    raw_data = response.json()
